In [ ]:
import datetime
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras import Sequential,activations
from keras.layers import Dense,LSTM,Reshape,Dropout,InputLayer,Flatten,Input,BatchNormalization
from keras.callbacks import TensorBoard , ModelCheckpoint
import pathlib

mpl.rcParams['figure.figsize'] = (12, 8)
mpl.rcParams['axes.grid'] = False

In [ ]:
df = pd.read_csv('data/sp100_indicator_1990.csv')
df.pop('Unnamed: 0')

date_time = pd.to_datetime(df.pop('Date'),
                           format='%Y.%m.%d')

timestamp_s = date_time.map(pd.Timestamp.timestamp)
df['Time']= timestamp_s
# timestamp_s = timestamp_s.map(pd.Timestamp.normalize)
# timestamp_s = pd.Timestamp(date_time)
df


In [ ]:
# timestamp_s = date_time.map(pd.Timestamp.timestamp)
# timestamp_s
# year = 365.2425
# df['1 Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
# df['1 Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

In [ ]:
# twoYear = year * 2
# threeYear = year * 3
# fourYear = year *4
# fiveYear = year *5
# sixYear = year *6
# sevenYear = year *7
# eightYear = year *8
# nineYear = year *9
# tenYear = year *10
# twentyYear = year *20
# fortyYear = year * 40

In [ ]:

# df['2 Year sin'] = np.sin(timestamp_s * (2 * np.pi / twoYear))
# df['2 Year cos'] = np.cos(timestamp_s * (2 * np.pi / twoYear))
# df['3 Year sin'] = np.sin(timestamp_s * (2 * np.pi / threeYear))
# df['3 Year cos'] = np.cos(timestamp_s * (2 * np.pi / threeYear))
# df['4 Year sin'] = np.sin(timestamp_s * (2 * np.pi / fourYear))
# df['4 Year cos'] = np.cos(timestamp_s * (2 * np.pi / fourYear))
# df['5 Year sin'] = np.sin(timestamp_s * (2 * np.pi / fiveYear))
# df['5 Year cos'] = np.cos(timestamp_s * (2 * np.pi / fiveYear))
# df['6 Year sin'] = np.sin(timestamp_s * (2 * np.pi / sixYear))
# df['6 Year cos'] = np.cos(timestamp_s * (2 * np.pi / sixYear))
# df['7 Year sin'] = np.sin(timestamp_s * (2 * np.pi / sevenYear))
# df['7 Year cos'] = np.cos(timestamp_s * (2 * np.pi / sevenYear))
# df['8 Year sin'] = np.sin(timestamp_s * (2 * np.pi / eightYear))
# df['8 Year cos'] = np.cos(timestamp_s * (2 * np.pi / eightYear))
# df['9 Year sin'] = np.sin(timestamp_s * (2 * np.pi / nineYear))
# df['9 Year cos'] = np.cos(timestamp_s * (2 * np.pi / nineYear))
# df['10 Year sin'] = np.sin(timestamp_s * (2 * np.pi / tenYear))
# df['10 Year cos'] = np.cos(timestamp_s * (2 * np.pi / tenYear))
# df['20 Year sin'] = np.sin(timestamp_s * (2 * np.pi / twentyYear))
# df['20 Year cos'] = np.cos(timestamp_s * (2 * np.pi / twentyYear))
# df['40 Year sin'] = np.sin(timestamp_s * (2 * np.pi / fortyYear))
# df['40 Year cos'] = np.cos(timestamp_s * (2 * np.pi / fortyYear))

# df.head()

In [ ]:
# plot_cols = ['1 Year sin','2 Year sin','3 Year sin','4 Year sin','5 Year sin','6 Year sin','7 Year sin','8 Year sin','9 Year sin','10 Year sin','20 Year sin','40 Year sin']
# plot_features = df[plot_cols]
# plot_features.index = date_time
# _ = plot_features.plot(subplots=True)

In [ ]:
# fft = tf.signal.rfft(df['Close'])
# f_per_dataset = np.arange(0, len(fft))

# n_samples_h = len(df['Close'])

# hours_per_year = 365.2524
# years_per_dataset = n_samples_h/(hours_per_year)

# f_per_year = f_per_dataset/years_per_dataset
# plt.step(f_per_year, np.abs(fft))
# plt.xscale('log')

# _ = plt.xlabel('Frequency')

In [ ]:
class dataProcessing():

    def __init__(self ,data,input_width ,prediction_days , label_columns , stockname):

        self.stockname = stockname
        self.input_width = input_width
        self.prediction_days = prediction_days
        self.label_columns = label_columns
        self.data = data

        self.column_indices = {name: i for i, name in enumerate(data.columns)}
        self.num_features = data.shape[1]
        
        #slit into test,validation and train data
        n = len(data)
        self.train_dataset = data[:int(0.95 * n)]
        # self.val_dataset = data[-200:-100]
        self.test_dataset = data[int(0.95 * n):]
        
        #reset indecies

        # self.val_dataset = self.val_dataset.reset_index()
        # self.val_dataset.pop('index')
        self.test_dataset = self.test_dataset.reset_index()
        self.test_dataset.pop('index')
        

        #Normalizing The Data

        self.train_std = self.train_dataset.std()  
        self.train_mean = self.train_dataset.mean()
        
        self.train_dataset = (self.train_dataset - self.train_mean) / self.train_std
        # self.val_dataset = (self.val_dataset - self.train_mean) / self.train_std
        self.test_dataset = (self.test_dataset - self.train_mean) / self.train_std

    def plot_normalized_data(self):
        data_std = (self.data - self.train_mean) / self.train_std
        data_std = data_std.melt(var_name='Column', value_name='Normalized')
        plt.figure(figsize=(12, 6))
        ax = sns.violinplot(x='Column', y='Normalized', data=data_std)
        _ = ax.set_xticklabels(self.data.keys(), rotation=90)

    def make_windows(self,data:pd.core.frame.DataFrame):
        
        window_input = []
        window_output=[]

        for i in range(self.input_width,len(data)-self.prediction_days):

            
            window_input.append(data[i-self.input_width:i].reset_index())
            window_output.append(data[self.label_columns][i:i+self.prediction_days].reset_index())

            window_input[-1].pop('index')
            window_output[-1].pop('index')
            
            #convert pd.DataFrame to numpy
            window_input[-1]= window_input[-1].to_numpy() 
            window_output[-1]= window_output[-1].to_numpy() 

        #convert list to numpy
        window_input = np.asarray(window_input)
        window_output = np.asarray(window_output)

        return window_input,window_output  

    def test(self, model,test_data):

        x_test , y_test = self.make_windows(test_data)

        predicted_prices=model.predict(x_test)

        for i in range(len(self.label_columns)):
            predicted_prices[:,:,i]=(predicted_prices[:,:,i]*self.train_std[self.label_columns[i]])+self.train_mean[self.label_columns[i]]
            y_test[:,:,i]=(y_test[:,:,i]*self.train_std[self.label_columns[i]])+self.train_mean[self.label_columns[i]]        

        return predicted_prices , y_test
        

    # def test(self,model,test_data):

    #     predicted_prices=model.predict(test_data)

    #     for i in range(len(self.label_columns)):
    #         predicted_prices[:,:,i]=(predicted_prices[:,:,i]*self.train_std[self.label_columns[i]])+self.train_mean[self.label_columns[i]]
    #         y_test[:,:,i]=(y_test[:,:,i]*self.train_std[self.label_columns[i]])+self.train_mean[self.label_columns[i]]        

    #     return predicted_prices , y_test
        


    def plot_and_saveCSV(self,model,modelname,predicted,real):

        #check if path is available
        path = f'./models/{modelname}/{self.stockname}/figures'
        pathlib.Path(path).mkdir(parents=True,exist_ok=True)

        # pd.DataFrame(predicted).to_csv(f'models/{modelname}/{self.stockname}/predicted.csv')
        # pd.DataFrame(real).to_csv(f'models/{modelname}/{self.stockname}/real.csv')

        for i in range(len(real)):
                
            plt.plot(real[i,:,:], color="black",label=f"real", marker='o', ms=10)
            plt.plot(predicted[i,:,:], color="green",label=f"predicted", marker='o', ms=10)

            plt.savefig(f'models/{modelname}/{self.stockname}/figures/fig-{i+1}.png')
            plt.clf()
        


In [ ]:
class WindowGenerator():

    def __init__(self, input_width, label_width, shift,
                train_df, val_df, test_df,label_columns=None):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                                enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]
        
    def __repr__(self):
        return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])


    def split_window(self, features):   
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels

    def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
        inputs, labels = self.example
        plt.figure(figsize=(12, 8))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))
        for n in range(max_n):
            plt.subplot(max_n, 1, n+1)
            plt.ylabel(f'{plot_col} [normed]')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                    label='Inputs', marker='.', zorder=-10)

            if self.label_columns:
                label_col_index = self.label_columns_indices.get(plot_col, None)
            else:
                label_col_index = plot_col_index

            if label_col_index is None:
                continue

            plt.scatter(self.label_indices, labels[n, :, label_col_index],
                        edgecolors='k', label='Labels', c='#2ca02c', s=64)
            if model is not None:
                predictions = model(inputs)
                plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                            marker='X', edgecolors='k', label='Predictions',
                            c='#ff7f0e', s=64)

            if n == 0:
                plt.legend()

        plt.xlabel('Time [h]')

    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=128,)

        ds = ds.map(self.split_window)

        return ds

    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        # """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
        # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
        # And cache it for next time
            self._example = result
        return result 

In [ ]:
def compile_and_fit(model,modelname,data:dataProcessing,window:WindowGenerator):
    MAX_EPOCHS = 50

    #check if path is available
    path = f'models/{modelname}/{data.stockname}/tensorboard/logs/fit'
    pathlib.Path(path).mkdir(parents=True,exist_ok=True)

    filepath = f'models/{modelname}/{data.stockname}'
    
    model_checkpoint_callback= tf.keras.callbacks.ModelCheckpoint(
        filepath, monitor='loss', verbose=0, save_best_only=True,
        save_weights_only=False, mode='auto', save_freq='epoch',
        options=None
    )

    log_dir =f'models/{modelname}/{data.stockname}/tensorboard/logs/fit/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


    model.compile(loss=tf.losses.MeanSquaredError(),
                    optimizer=tf.optimizers.Adam(),
                    metrics=[tf.metrics.MeanAbsoluteError(),tf.metrics.MeanAbsolutePercentageError()])

    history = model.fit(window , epochs=MAX_EPOCHS,verbose=1,callbacks=[tensorboard_callback,model_checkpoint_callback])
    
    return history

In [ ]:
data = dataProcessing(df,input_width=60,prediction_days=1,label_columns=['Close'],stockname='sp100_with_Indicator')


In [ ]:
wg = WindowGenerator(  
                        input_width=data.input_width,
                        shift=data.prediction_days,
                        label_width=1,
                        train_df=data.train_dataset,
                        val_df=None,
                        test_df=data.test_dataset,
                        label_columns=['Close']
                    )


In [ ]:
iw , ow = data.make_windows(data.train_dataset)

In [ ]:
fitures = data.num_features
days= data.input_width

test_model = Sequential()

test_model.add(LSTM(units=fitures, input_shape = (days,fitures)))
test_model.add(Dense(1000,activation=activations.elu))

test_model.add(Flatten())
test_model.add(Dense(units=1))
test_model.add(Reshape([1,1]))
test_model.summary()




In [ ]:
history= compile_and_fit(
                            model=test_model,
                            modelname='test_model',
                            data=data,
                            window = wg.train
                        )


In [ ]:

input_T ,real = data.make_windows(data.test_dataset)
predictions = test_model.predict(input_T)

list_a = [i for i in real[:,0,0]]

list_real = [i*data.train_std['Close']+data.train_mean['Close'] for i in list_a]

list_a = [i for i in predictions[:,0,0]]

list_predict = [i*data.train_std['Close']+data.train_mean['Close'] for i in list_a]

plt.plot(list_real, color="black",label=f"real", marker='o', ms=1)
plt.plot(list_predict, color="green",label=f"predicted", marker='o', ms=1)